In [11]:
from ysdc_dataset_api.dataset import MotionPredictionDataset

In [12]:
# Load protobufs for training dataset
dataset_path = '/media/cds-k/Data_2/canonical-trn-dev-data/data/train_pb/'

# Path to file with training scene tags
scene_tags_fpath = '/media/cds-k/Data_2/canonical-trn-dev-data/data/train_tags.txt'

In [13]:
import json
with open('/home/cds-k/Desktop/motion_prediction/Trajectron-plus-plus/experiments/nuScenes/train_configs/config_ph_25_maxhl_24_minhl_24.json', 'r', encoding='utf-8') as conf_json:
    hyperparams = json.load(conf_json)

In [14]:
import numpy as np

standardization = {
    'PEDESTRIAN': {
        'position': {
            'x': {'mean': 0, 'std': 1},
            'y': {'mean': 0, 'std': 1}
        },
        'velocity': {
            'x': {'mean': 0, 'std': 2},
            'y': {'mean': 0, 'std': 2}
        },
        'acceleration': {
            'x': {'mean': 0, 'std': 1},
            'y': {'mean': 0, 'std': 1}
        }
    },
    'VEHICLE': {
        'position': {
            'x': {'mean': 0, 'std': 80},
            'y': {'mean': 0, 'std': 80}
        },
        'velocity': {
            'x': {'mean': 0, 'std': 15},
            'y': {'mean': 0, 'std': 15},
            'norm': {'mean': 0, 'std': 15}
        },
        'acceleration': {
            'x': {'mean': 0, 'std': 4},
            'y': {'mean': 0, 'std': 4},
            'norm': {'mean': 0, 'std': 4}
        },
        'heading': {
            'x': {'mean': 0, 'std': 1},
            'y': {'mean': 0, 'std': 1},
            '°': {'mean': 0, 'std': np.pi},
            'd°': {'mean': 0, 'std': 1}
        }
    }
}

In [15]:
import sys
sys.path.append("../../../trajectron")
from environment import Environment, Scene, Node
from environment import derivative_of as derivative_of
env = Environment(node_type_list=['VEHICLE', 'PEDESTRIAN'], standardization=standardization)
attention_radius = dict()
attention_radius[(env.NodeType.PEDESTRIAN, env.NodeType.PEDESTRIAN)] = 10.0
attention_radius[(env.NodeType.PEDESTRIAN, env.NodeType.VEHICLE)] = 20.0
attention_radius[(env.NodeType.VEHICLE, env.NodeType.PEDESTRIAN)] = 20.0
attention_radius[(env.NodeType.VEHICLE, env.NodeType.VEHICLE)] = 30.0

env.attention_radius = attention_radius


In [47]:
env.NodeType

[VEHICLE, PEDESTRIAN]

In [17]:
dataset = MotionPredictionDataset(
    dataset_path=dataset_path,
    scene_tags_fpath=scene_tags_fpath,
    feature_producer=None,
    transform_ground_truth_to_agent_frame=True,
    hyperparams=hyperparams,
    node_type=env.NodeType[0]
)

388406/388406 scenes fit the filter criteria.


In [18]:
dataset_iter = iter(dataset)

In [19]:
for i in range(1):
    data_item = next(dataset_iter)

In [20]:
data_item[2].shape

torch.Size([25, 2])

In [29]:
import collections.abc
from torch.utils.data._utils.collate import default_collate
import dill
container_abcs = collections.abc

In [30]:
def collate(batch):
    if len(batch) == 0:
        return batch
    elem = batch[0]
    if elem is None:
        return None
    elif isinstance(elem, container_abcs.Sequence):
        if len(elem) == 4: # We assume those are the maps, map points, headings and patch_size
            scene_map, scene_pts, heading_angle, patch_size = zip(*batch)
            if heading_angle[0] is None:
                heading_angle = None
            else:
                heading_angle = torch.Tensor(heading_angle)
            map = scene_map[0].get_cropped_maps_from_scene_map_batch(scene_map,
                                                                     scene_pts=torch.Tensor(scene_pts),
                                                                     patch_size=patch_size[0],
                                                                     rotation=heading_angle)
            return map
        transposed = zip(*batch)
        return [collate(samples) for samples in transposed]
    elif isinstance(elem, container_abcs.Mapping):
        # We have to dill the neighbors structures. Otherwise each tensor is put into
        # shared memory separately -> slow, file pointer overhead
        # we only do this in multiprocessing
        neighbor_dict = {key: [d[key] for d in batch] for key in elem}
        return dill.dumps(neighbor_dict) if torch.utils.data.get_worker_info() else neighbor_dict
    return default_collate(batch)

In [31]:
import torch

dataloader_kwargs = {
    'batch_size': 5
}


dataloader = torch.utils.data.DataLoader(dataset, batch_size=5, collate_fn=collate)

In [32]:
for batch in dataloader:
    print(batch)
    break

[tensor([0, 0, 0, 0, 0]), tensor([[[ 7.4081e+01,  1.0634e+02, -5.8606e+00,  1.9477e+01,  0.0000e+00,
           0.0000e+00,  1.8333e+00,  8.8353e-02],
         [ 7.2908e+01,  1.1023e+02, -5.8606e+00,  1.9477e+01,  0.0000e+00,
           0.0000e+00,  1.8510e+00,  8.8353e-02],
         [ 7.1651e+01,  1.1412e+02, -6.2895e+00,  1.9425e+01, -2.1449e+00,
          -2.6303e-01,  1.8737e+00,  1.1340e-01],
         [ 7.0223e+01,  1.1798e+02, -7.1361e+00,  1.9321e+01, -4.2327e+00,
          -5.1914e-01,  1.8984e+00,  1.2397e-01],
         [ 6.8918e+01,  1.2175e+02, -6.5245e+00,  1.8811e+01,  3.0580e+00,
          -2.5510e+00,  1.9267e+00,  1.4149e-01],
         [ 6.7742e+01,  1.2553e+02, -5.8817e+00,  1.8905e+01,  3.2138e+00,
           4.7436e-01,  1.8529e+00, -3.6937e-01],
         [ 6.6269e+01,  1.2929e+02, -7.3662e+00,  1.8805e+01, -7.4223e+00,
          -4.9998e-01,  1.8612e+00,  4.1798e-02],
         [ 6.4855e+01,  1.3299e+02, -7.0699e+00,  1.8514e+01,  1.4812e+00,
          -1.4559e+00,  

In [33]:
batch[0]

tensor([0, 0, 0, 0, 0])

In [35]:
batch[1].shape

torch.Size([5, 25, 8])

In [36]:
batch[2].shape

torch.Size([5, 25, 2])

In [39]:
batch[3].shape

torch.Size([5, 25, 8])

In [40]:
batch[4].shape

torch.Size([5, 25, 2])

In [42]:
batch[5]

{}

In [43]:
batch[6]

{}

In [44]:
batch[7]

In [45]:
batch[8]